**Loading the data**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Copy of twitter_training.csv")

In [ ]:
df.head()

,2401,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


Removing unwanted columns

In [ ]:
df = df.drop(["2401","Borderlands"],axis=1)

Renaming the columns

In [ ]:
df.columns = ["Sentiment","Comments"]

In [ ]:
df.head()

,Sentiment,Comments
0,Positive,I am coming to the borders and I will kill you...
1,Positive,im getting on borderlands and i will kill you ...
2,Positive,im coming on borderlands and i will murder you...
3,Positive,im getting on borderlands 2 and i will murder ...
4,Positive,im getting into borderlands and i can murder y...


Removing the Null records

In [ ]:
df = df.dropna(axis=0)

USE(Universal Sentence Encoder) is encoder to generate embeddings from given text and can load it using tensorflow_hub and in the embed function input text or entered text is passed to the USE and embeddings are returned each text is represented by a vector of 512 numbers.

In [ ]:
import tensorflow_hub as hub
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)


module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


 All labels are just converted to numbers and are stored in the list

In [ ]:
sen = []
for i in df.Sentiment:
    if i == "Positive":
      sen.append(0)
    elif i=="Neutral":
      sen.append(1)
    else:
      sen.append(2)

In [ ]:
num_labels = sen

In [ ]:
len(sen)

73995

Converting the list of numbers into one hot labels

In [ ]:
from keras.utils import to_categorical

In [ ]:
sen = to_categorical(sen,num_classes=3)

Creating the model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Input,Flatten

In [ ]:
NN = Sequential([
    Input(shape=(512,)),
    Dense(512,activation="relu"),
    Dense(512,activation="relu"),
    Dense(512,activation="relu"),
    Dense(3,activation = "softmax")
])

In [ ]:
embed(["hi"]).numpy().shape

(1, 512)

Embeddings are generated for all text.

In [ ]:
text_embed = []
for i in df.Comments:
  a = embed([i])
  text_embed.append(a[0].numpy())

Splitting the data into training data and testing data

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
xtrain,xtest,ytrain,ytest = train_test_split(text_embed,sen,test_size=.2)

Compiling and Training the model

In [ ]:
NN.compile(loss='categorical_crossentropy',optimizer='adam',metrics=["accuracy"])

In [ ]:
NN.fit(np.array(xtrain),ytrain,epochs=10,batch_size=16,validation_data=(np.array(xtest),ytest))

Epoch 1/10
3700/3700 [==============================] - 44s 11ms/step - loss: 0.7452 - accuracy: 0.6780 - val_loss: 0.6573 - val_accuracy: 0.7205
Epoch 2/10
3700/3700 [==============================] - 43s 12ms/step - loss: 0.4671 - accuracy: 0.8145 - val_loss: 0.4723 - val_accuracy: 0.8104
Epoch 3/10
3700/3700 [==============================] - 42s 11ms/step - loss: 0.2780 - accuracy: 0.8914 - val_loss: 0.4482 - val_accuracy: 0.8321
Epoch 4/10
3700/3700 [==============================] - 43s 11ms/step - loss: 0.1897 - accuracy: 0.9261 - val_loss: 0.4568 - val_accuracy: 0.8444
Epoch 5/10
3700/3700 [==============================] - 42s 11ms/step - loss: 0.1499 - accuracy: 0.9407 - val_loss: 0.4643 - val_accuracy: 0.8536
Epoch 6/10
3700/3700 [==============================] - 43s 12ms/step - loss: 0.1267 - accuracy: 0.9497 - val_loss: 0.5026 - val_accuracy: 0.8521
Epoch 7/10
3700/3700 [==============================] - 42s 11ms/step - loss: 0.1090 - accuracy: 0.9569 - val_loss: 0.5381 -

Saving Model


In [ ]:
NN.save('/content/drive/MyDrive/text_classification_NN.keras')

In [ ]:
d = NN(embed(["A negative mind will never give you a positive life."]).numpy())

In [ ]:
np.array(d)

array([[0.0399875 , 0.00107225, 0.95894027]], dtype=float32)